In [129]:
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer

import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer

from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle


# using the SQLite Table to read data.
con = sqlite3.connect('../input/database.sqlite') 

#filtering only positive and negative reviews i.e. 
# not taking into consideration those reviews with Score=3
filtered_data = pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3
""", con) 

# Give reviews with Score>3 a positive rating, and reviews with a score<3 a negative rating.
def partition(x):
    if x < 3:
        return 'negative'
    return 'positive'

#changing reviews with score less than 3 to be positive and vice-versa
actualScore = filtered_data['Score']
positiveNegative = actualScore.map(partition) 
filtered_data['Score'] = positiveNegative

In [130]:
print(filtered_data.shape) #looking at the number of attributes and size of the data
filtered_data.head()

(525814, 10)


,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,positive,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,negative,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,positive,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,negative,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,positive,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [131]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND UserId="AR5J8UI46CURR"
ORDER BY ProductID
""", con)
display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,78445,B000HDL1RQ,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
1,138317,B000HDOPYC,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
2,138277,B000HDOPYM,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
3,73791,B000HDOPZG,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...
4,155049,B000PAQ75C,AR5J8UI46CURR,Geetha Krishnan,2,2,5,1199577600,LOACKER QUADRATINI VANILLA WAFERS,DELICIOUS WAFERS. I FIND THAT EUROPEAN WAFERS ...


In [132]:
#Sorting data according to ProductId in ascending order
sorted_data=filtered_data.sort_values('ProductId', axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')

In [133]:
#Deduplication of entries
final=sorted_data.drop_duplicates(subset={"UserId","ProfileName","Time","Text"}, keep='first', inplace=False)
final.shape

(364173, 10)

In [134]:
#Checking to see how much % of data still remains
(final['Id'].size*1.0)/(filtered_data['Id'].size*1.0)*100

69.25890143662969

In [135]:
display= pd.read_sql_query("""
SELECT *
FROM Reviews
WHERE Score != 3 AND Id=44737 OR Id=64422
ORDER BY ProductID
""", con)

display.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,64422,B000MIDROQ,A161DK06JJMCYF,"J. E. Stephens ""Jeanne""",3,1,5,1224892800,Bought This for My Son at College,My son loves spaghetti so I didn't hesitate or...
1,44737,B001EQ55RW,A2V0I904FH7ABY,Ram,3,2,4,1212883200,Pure cocoa taste with crunchy almonds inside,It was almost a 'love at first bite' - the per...


In [136]:
final=final[final.HelpfulnessNumerator<=final.HelpfulnessDenominator]

In [137]:
#Before starting the next phase of preprocessing lets see the number of entries left
print(final.shape)

#How many positive and negative reviews are present in our dataset?
final['Score'].value_counts()

(364171, 10)


positive    307061
negative     57110
Name: Score, dtype: int64

In [138]:
# find sentences containing HTML tags
import re
i=0;
for sent in final['Text'].values:
    if (len(re.findall('<.*?>', sent))):
        print(i)
        print(sent)
        break;
    i += 1;

6
I set aside at least an hour each day to read to my son (3 y/o). At this point, I consider myself a connoisseur of children's books and this is one of the best. Santa Clause put this under the tree. Since then, we've read it perpetually and he loves it.<br /><br />First, this book taught him the months of the year.<br /><br />Second, it's a pleasure to read. Well suited to 1.5 y/o old to 4+.<br /><br />Very few children's books are worth owning. Most should be borrowed from the library. This book, however, deserves a permanent spot on your shelf. Sendak's best.


In [139]:
stop = set(stopwords.words('english')) #set of stopwords
sno = nltk.stem.SnowballStemmer('english') #initialising the snowball stemmer

def cleanhtml(sentence): #function to clean the word of any html-tags
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, ' ', sentence)
    return cleantext
def cleanpunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    return  cleaned
print(stop)
print('************************************')
print(sno.stem('tasty'))

{'the', "wasn't", 'them', 'yourself', 'mustn', 'and', 'don', 's', 'very', 'most', 'at', 'while', 'does', 'that', 'her', 'did', 'before', 'am', 'but', 'if', 'yourselves', 'what', 'those', "don't", "haven't", 'or', 'will', "needn't", "won't", 'against', 'y', 'this', 'their', 'll', 'there', 'aren', 'has', 'each', 'shan', 'ain', 'just', 'him', 'below', 'until', 'it', 'm', 'ourselves', 'nor', 'which', 'in', 'about', 'should', 'weren', 'are', 'isn', 'hasn', 'me', "you'd", 'is', 'with', 'further', 'so', 've', 'because', 'on', 'itself', 'didn', "didn't", 'mightn', 'herself', 'you', 'between', 'only', 'other', 'wasn', "shouldn't", 'they', 'all', 'its', 'why', 'once', 'through', 'theirs', 'when', 'no', "weren't", 'few', 'your', 'over', 'were', 'then', "hadn't", "wouldn't", 'as', 'up', 'ma', 'same', "it's", 'down', 'o', 'how', 'd', 'than', 'to', 'had', 'these', 're', 'having', "shan't", 'myself', 'own', "mustn't", 'hadn', "she's", 't', "couldn't", 'ours', 'won', 'both', 'from', 'his', 'of', 'he',

In [140]:
#Code for implementing step-by-step the checks mentioned in the pre-processing phase
# this code takes a while to run as it needs to run on 500k sentences.
i=0
str1=' '
final_string=[]
all_positive_words=[] # store words from +ve reviews here
all_negative_words=[] # store words from -ve reviews here.
s=''
for sent in final['Text'].values:
    filtered_sentence=[]
    #print(sent);
    sent=cleanhtml(sent) # remove HTMl tags
    for w in sent.split():
        for cleaned_words in cleanpunc(w).split():
            if((cleaned_words.isalpha()) & (len(cleaned_words)>2)):    
                if(cleaned_words.lower() not in stop):
                    s=(sno.stem(cleaned_words.lower())).encode('utf8')
                    filtered_sentence.append(s)
                    if (final['Score'].values)[i] == 'positive': 
                        all_positive_words.append(s) #list of all words used to describe positive reviews
                    if(final['Score'].values)[i] == 'negative':
                        all_negative_words.append(s) #list of all words used to describe negative reviews reviews
                else:
                    continue
            else:
                continue 
    #print(filtered_sentence)
    str1 = b" ".join(filtered_sentence) #final string of cleaned words
    #print("***********************************************************************")
    
    final_string.append(str1)
    i+=1

In [141]:
final['CleanedText']=final_string #adding a column of CleanedText which displays the data after pre-processing of the review 
final['CleanedText']=final['CleanedText'].str.decode("utf-8")

In [142]:
final.head(3) #below the processed review can be seen in the CleanedText Column 


# store final table into an SQlLite table for future.
conn = sqlite3.connect('final.sqlite')
c=conn.cursor()
conn.text_factory = str
final.to_sql('Reviews', conn,  schema=None, if_exists='replace', index=True, index_label=None, chunksize=None, dtype=None)

In [143]:
final.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
138706,150524,0006641040,ACITT7DI6IDDL,shari zychinski,0,0,positive,939340800,EVERY book is educational,this witty little book makes my son laugh at l...,witti littl book make son laugh loud recit car...
138688,150506,0006641040,A2IW4PEEKO2R0U,Tracy,1,1,positive,1194739200,"Love the book, miss the hard cover version","I grew up reading these Sendak books, and watc...",grew read sendak book watch realli rosi movi i...
138689,150507,0006641040,A1S4A3IQ2MU7V4,"sally sue ""sally sue""",1,1,positive,1191456000,chicken soup with rice months,This is a fun way for children to learn their ...,fun way children learn month year learn poem t...
138690,150508,0006641040,AZGXZ2UUK6X,"Catherine Hallberg ""(Kate)""",1,1,positive,1076025600,a good swingy rhythm for reading aloud,This is a great little book to read aloud- it ...,great littl book read nice rhythm well good re...
138691,150509,0006641040,A3CMRKGE0P909G,Teresa,3,4,positive,1018396800,A great way to learn the months,This is a book of poetry about the months of t...,book poetri month year goe month cute littl po...


In [144]:
final['Time'] = pd.to_datetime(final['Time'])

In [145]:
# Random sampling
df = final.take(np.random.permutation(len(final))[:8000])

In [146]:
# Sort by time
data = df.sort_values(by='Time')
data['Score'] = data['Score'].map({"positive":1, "negative":0})
data.head(2)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,CleanedText
156021,169214,B0001ES9FI,A2WE1DWPQRDBFW,Scotbuff,6,6,1,1970-01-01 00:00:01.088035200,Full Bodied and Never Burnt,The Dark Roast is the best of the coffee pods ...,dark roast best coffe pod avail senseo coffe m...
156032,169228,B0001ES9FI,A23ZRFLO4FCVXE,"Diane Houslanger ""JapaRider""",1,5,1,1970-01-01 00:00:01.089504000,Good coffee,"What more can be said? Good, clean, full bodie...",said good clean full bodi sound like commerci ...


# Bag of Words (BOW)

In [175]:
from sklearn.model_selection import train_test_split

# Spliting into Train and test
X_train, X_test, y_train, y_test = train_test_split(data['CleanedText'].values,data['Score'].values,test_size=0.3,shuffle=False)

In [176]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing

count_vect = CountVectorizer()  
X_train = count_vect.fit_transform(X_train)
print("Train Data Size: ",X_train.shape)
X_test = count_vect.transform(X_test)
print("Test Data Size: ",X_test.shape)

Train Data Size:  (5600, 10147)
Test Data Size:  (2400, 10147)


## KNN with 'brute' algorithm

In [178]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=9, algorithm='brute')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.8175
---------------------------------------------------------------------------------------------------------------------
[0.85561497 0.83244207 0.84670232 0.84107143 0.8375     0.84821429
 0.84107143 0.83542039 0.86046512 0.85867621]
---------------------------------------------------------------------------------------------------------------------
0.8258333333333333


## KNN with 'kd tree' algorithm

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='kd_tree')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=9, algorithm='kd_tree')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

# TF-IDF

In [153]:
from sklearn.model_selection import train_test_split

# Spliting into Train and test
X_train, X_test, y_train, y_test = train_test_split(data['CleanedText'].values,data['Score'].values,test_size=0.3,shuffle=False)

In [154]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

tfidf = TfidfVectorizer(ngram_range=(1,2)) 
X_train = tfidf.fit_transform(X_train)
#Normalize train Data
X_train = preprocessing.normalize(X_train)
print("Train Data Size: ",X_train.shape)
X_test = tfidf.transform(X_test)
#Normalize test Data
X_test = preprocessing.normalize(X_test)
print("Test Data Size: ",X_test.shape)

Train Data Size:  (5600, 148938)
Test Data Size:  (2400, 148938)


In [155]:
features = tfidf.get_feature_names()

## KNN with 'brute' algorithm

In [156]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=9, algorithm='brute')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.83125
---------------------------------------------------------------------------------------------------------------------
[0.84670232 0.83778966 0.85561497 0.85535714 0.85535714 0.83571429
 0.85178571 0.8372093  0.85867621 0.85330948]
---------------------------------------------------------------------------------------------------------------------
0.8341666666666666


## KNN with 'kd tree' algorithm

In [157]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='kd_tree')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=9, algorithm='kd_tree')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.83125
---------------------------------------------------------------------------------------------------------------------
[0.84670232 0.83778966 0.85561497 0.85535714 0.85535714 0.83571429
 0.85178571 0.8372093  0.85867621 0.85330948]
---------------------------------------------------------------------------------------------------------------------
0.8341666666666666


 # W2V

In [158]:
# Train your own Word2Vec model using your own text corpus
i=0
list_of_sent=[]
for sent in df['CleanedText'].values:
    list_of_sent.append(sent.split())

In [159]:
print(df['CleanedText'].values[0])
print("*****************************************************************")
print(list_of_sent[0])

bought scharffen berger milk chocol bar eaten enjoy compani chocol bar new guinea chocol base ive eaten one thus far resist temptat eat remain five one sit although real treat expens get real satisfact
*****************************************************************
['bought', 'scharffen', 'berger', 'milk', 'chocol', 'bar', 'eaten', 'enjoy', 'compani', 'chocol', 'bar', 'new', 'guinea', 'chocol', 'base', 'ive', 'eaten', 'one', 'thus', 'far', 'resist', 'temptat', 'eat', 'remain', 'five', 'one', 'sit', 'although', 'real', 'treat', 'expens', 'get', 'real', 'satisfact']


In [160]:
# min_count = 5 considers only words that occured atleast 5 times
w2v_model = Word2Vec(list_of_sent,min_count=5,size=50, workers=4)

In [161]:
w2v_words = list(w2v_model.wv.vocab)
print("number of words that occured minimum 5 times ",len(w2v_words))
print("sample words ", w2v_words[0:50])

number of words that occured minimum 5 times  4081
sample words  ['bought', 'scharffen', 'berger', 'milk', 'chocol', 'bar', 'eaten', 'enjoy', 'compani', 'new', 'base', 'ive', 'one', 'thus', 'far', 'resist', 'temptat', 'eat', 'remain', 'five', 'sit', 'although', 'real', 'treat', 'expens', 'get', 'satisfact', 'dog', 'finicki', 'wont', 'can', 'food', 'love', 'make', 'great', 'topper', 'kibbl', 'two', 'type', 'varieti', 'pack', 'pork', 'beef', 'thick', 'pasti', 'good', 'hide', 'pill', 'chicken', 'mix']


In [162]:
count_vect_feat = count_vect.get_feature_names() # list of words in the BoW
print(count_vect_feat[count_vect_feat.index('like')])

like


# Average Word2Vec

In [163]:
# average Word2Vec
# compute average word2vec for each review.
sent_vectors = []; # the avg-w2v for each sentence/review is stored in this list
for sent in list_of_sent: # for each review/sentence
    sent_vec = np.zeros(50) # as word vectors are of zero length
    cnt_words =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            sent_vec += vec
            cnt_words += 1
    if cnt_words != 0:
        sent_vec /= cnt_words
    sent_vectors.append(sent_vec)
print(len(sent_vectors))
print(len(sent_vectors[0]))

8000
50


In [164]:
from sklearn.model_selection import train_test_split

# Spliting into Train and test
X_train, X_test, y_train, y_test = train_test_split(sent_vectors,data['Score'].values,test_size=0.3,shuffle=False)

## KNN with 'brute' algorithm

In [166]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=7, algorithm='brute')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.7825
---------------------------------------------------------------------------------------------------------------------
[0.82174688 0.8057041  0.83778966 0.81964286 0.80892857 0.83035714
 0.83571429 0.8157424  0.82468694 0.82289803]
---------------------------------------------------------------------------------------------------------------------
0.8216666666666667


## KNN with 'kd tree' algorithm

In [168]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='kd_tree')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=7, algorithm='kd_tree')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.7825
---------------------------------------------------------------------------------------------------------------------
[0.82174688 0.8057041  0.83778966 0.81964286 0.80892857 0.83035714
 0.83571429 0.8157424  0.82468694 0.82289803]
---------------------------------------------------------------------------------------------------------------------
0.8216666666666667


# TF-IDF weighted Word2Vec

In [169]:
tf_idf_vect = TfidfVectorizer(ngram_range=(1,2))
final_tf_idf = tf_idf_vect.fit_transform(df['CleanedText'].values)
print("the type of count vectorizer ",type(final_tf_idf))
print("the shape of out text TFIDF vectorizer ",final_tf_idf.get_shape())
print("the number of unique words including both unigrams and bigrams ", final_tf_idf.get_shape()[1])

the type of count vectorizer  <class 'scipy.sparse.csr.csr_matrix'>
the shape of out text TFIDF vectorizer  (8000, 197851)
the number of unique words including both unigrams and bigrams  197851


In [170]:
# TF-IDF weighted Word2Vec
tfidf_feat = tf_idf_vect.get_feature_names() # tfidf words/col-names
# final_tf_idf is the sparse matrix with row= sentence, col=word and cell_val = tfidf

tfidf_sent_vectors = []; # the tfidf-w2v for each sentence/review is stored in this list
row=0;
for sent in list_of_sent: # for each review/sentence 
    sent_vec = np.zeros(50) # as word vectors are of zero length
    weight_sum =0; # num of words with a valid vector in the sentence/review
    for word in sent: # for each word in a review/sentence
        if word in w2v_words:
            vec = w2v_model.wv[word]
            # obtain the tf_idfidf of a word in a sentence/review
            tf_idf = final_tf_idf[row, tfidf_feat.index(word)]
            sent_vec += (vec * tf_idf)
            weight_sum += tf_idf
    if weight_sum != 0:
        sent_vec /= weight_sum
    tfidf_sent_vectors.append(sent_vec)
    row += 1

In [171]:
print(len(tfidf_sent_vectors))
print(len(tfidf_sent_vectors[0]))

8000
50


In [172]:
from sklearn.model_selection import train_test_split

# Spliting into Train and test
X_train, X_test, y_train, y_test = train_test_split(tfidf_sent_vectors,data['Score'].values,test_size=0.3,shuffle=False)

## KNN with 'brute' algorithm

In [173]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='brute')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=9, algorithm='brute')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.79375
---------------------------------------------------------------------------------------------------------------------
[0.81996435 0.81639929 0.82531194 0.81071429 0.83035714 0.81964286
 0.81607143 0.82110912 0.82110912 0.82826476]
---------------------------------------------------------------------------------------------------------------------
0.8229166666666666


## KNN with 'kd tree' algorithm

In [174]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# KNN
knn = KNeighborsClassifier(n_neighbors=3, algorithm='kd_tree')
knn.fit(X_train, y_train)
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

print("---------------------------------------------------------------------------------------------------------------------")

# 10-fold Cross Validation
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(knn, X_train, y_train, cv=10)
print(cv_scores)

print("---------------------------------------------------------------------------------------------------------------------")

# KNN
knn = KNeighborsClassifier(n_neighbors=9, algorithm='kd_tree')
knn.fit(X_train, y_train)

#Accuracy on Test data
predicted_values = knn.predict(X_test)
score = accuracy_score(y_test, predicted_values)
print(score)

0.79375
---------------------------------------------------------------------------------------------------------------------
[0.81996435 0.81639929 0.82531194 0.81071429 0.83035714 0.81964286
 0.81607143 0.82110912 0.82110912 0.82826476]
---------------------------------------------------------------------------------------------------------------------
0.8229166666666666


# Observation
* By doing 10 fold cross validation we can see that, In both the algorithms ('kdtree' & 'brute') KNN is is taking K value between 5 to 10
* As I have taken only 8000 points, In that also I am getting pretty good accuracy score with all featurizing technique.